In [3]:
import tensorboard_reducer as tbr
import shutil
from glob import glob

In [4]:
input_event_dir=sorted(glob("lightning_logs/*"))
joined_dirs="\n".join(input_event_dir)
print(f"Found {len(input_event_dir)} runs:\n{joined_dirs}")

Found 12 runs:
lightning_logs\version_0
lightning_logs\version_1
lightning_logs\version_10
lightning_logs\version_11
lightning_logs\version_2
lightning_logs\version_3
lightning_logs\version_4
lightning_logs\version_5
lightning_logs\version_6
lightning_logs\version_7
lightning_logs\version_8
lightning_logs\version_9


In [5]:
joined_dirs

'lightning_logs\\version_0\nlightning_logs\\version_1\nlightning_logs\\version_10\nlightning_logs\\version_11\nlightning_logs\\version_2\nlightning_logs\\version_3\nlightning_logs\\version_4\nlightning_logs\\version_5\nlightning_logs\\version_6\nlightning_logs\\version_7\nlightning_logs\\version_8\nlightning_logs\\version_9'

In [14]:
del input_event_dir[0]


In [15]:
input_event_dir

['lightning_logs\\version_1',
 'lightning_logs\\version_10',
 'lightning_logs\\version_11',
 'lightning_logs\\version_2',
 'lightning_logs\\version_3',
 'lightning_logs\\version_4',
 'lightning_logs\\version_5',
 'lightning_logs\\version_6',
 'lightning_logs\\version_7',
 'lightning_logs\\version_8',
 'lightning_logs\\version_9']

In [16]:
del input_event_dir[3:8]

In [17]:
input_event_dir

['lightning_logs\\version_1',
 'lightning_logs\\version_10',
 'lightning_logs\\version_11',
 'lightning_logs\\version_7',
 'lightning_logs\\version_8',
 'lightning_logs\\version_9']

In [18]:
del input_event_dir[0]

In [23]:
events_out_dir = "tmp/reduced"
csv_out_path = "tmp/tb-reduction.csv"
overwrite = True
reduce_ops = ("mean", "min", "max")

events_dict = tbr.load_tb_events(input_event_dir,handle_dup_steps='keep-last')

n_scalars = len(events_dict)
n_steps, n_events = list(events_dict.values())[0].shape

print(
    f"Loaded {n_events} TensorBoard runs with {n_scalars} scalar(s) "
    f"and {n_steps} steps each:\n{events_dict}"
)

Loaded 5 TensorBoard runs with 6 scalar(s) and 1 steps each:
{'hp_metric':       value  value  value  value  value
step                                   
0      -1.0   -1.0   -1.0   -1.0   -1.0, 'lr-Adam':              value         value         value         value         value
step                                                                      
0     1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03
10    1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03
20    1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03
30    1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03
40    1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03
...            ...           ...           ...           ...           ...
1950  1.000000e-08  1.000000e-08  1.000000e-08  1.000000e-08  1.000000e-08
1960  1.000000e-08  1.000000e-08  1.000000e-08  1.000000e-08  1.000000e-08
1970  1.000000e-08  1.000000e-08  1.000000e

In [24]:
reduced_events = tbr.reduce_events(events_dict, reduce_ops)

for op in reduce_ops:
    print(f"Writing '{op}' reduction to '{events_out_dir}-{op}'")

tbr.write_tb_events(reduced_events, events_out_dir, overwrite)

print(f"Writing results to '{csv_out_path}'")

tbr.write_data_file(reduced_events, csv_out_path, overwrite)

print("✓ Reduction complete")

Writing 'mean' reduction to 'tmp/reduced-mean'
Writing 'min' reduction to 'tmp/reduced-min'
Writing 'max' reduction to 'tmp/reduced-max'
Writing results to 'tmp/tb-reduction.csv'
✓ Reduction complete


In [25]:
# clean up
shutil.rmtree("tmp")